In [19]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import sqlite3
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

def get_links():

    DRIVER_PATH = '/path/to/chromedriver'
    links_to_scrape=[]

    for page in range(1, 2):
        driver = webdriver.Chrome(executable_path=DRIVER_PATH)
        driver.get('https://asunnot.oikotie.fi/myytavatasunnot/helsinki?pagination='+ str(page))

        time.sleep(2)

        lnks = driver.find_elements(By.CLASS_NAME, "otcardv2")
        # traverse list
        for lnk in lnks:
            links_to_scrape.append(lnk.get_attribute("href"))
            print(lnk.get_attribute("href"))
    driver.quit()
    return links_to_scrape



def get_data(link):

    r = requests.get(link)
    
    soup = BeautifulSoup(r.content, 'html.parser')
    data=soup.select(".infotable__row")
        ##".infotable__title", ".infotable__value")
    #info=soup.select(".infotable__value")

    items={}
    for item in data:
        for i,j in zip(item.find_all("dt"), item.find_all("dd")):
            title=i.text.strip()
            value=j.text.strip()
            items[title]=value
            items['link']=link
            
    return items

def transform(df):

    columns_to_float=['Velaton hinta', 'Myyntihinta', 'Neliöhinta', 'Hoitovastike', 'Yhtiövastike', 'Vesimaksu']

    df=to_float(df, columns_to_float)

    df["pvm"]=date.today()
    df["id"]=df['link'].str[8:]
    df["status"]="aktiivinen"

    return df


def extract(links_to_scrape):
    df=pd.DataFrame()

    for link in links_to_scrape:
    ## append dict to df
        data=pd.DataFrame([get_data(link)])
        
        df=df.append(data)
    
    df=transform(df)

    return df



##Utils

def to_float(df, columns):

    for column in columns:
        replacement = {
        "/m2": "",
        "€": "",
        '\s+': '',
        ',':'.',
        '/kk': ''
        }

        df[column] = df[column].astype(str).str.replace(_, '').replace(replacement, regex=True).astype(float)

    return df

In [ ]:
links_to_scrape=get_links()

In [ ]:
df=pd.DataFrame()
for link in links_to_scrape:
## append dict to df
    data=pd.DataFrame([get_data(link)])
    df=df.append(data)

In [ ]:
df_all=extract(links_to_scrape)

In [64]:
def transform(df):



    df["pvm"]=date.today()
    df["id"]=df['link'].str[-8:]
    df["status"]="aktiivinen"

    df.columns = df.columns.str.replace('ä', 'a')
    df.columns = df.columns.str.replace('ö', 'o')
    df.columns = df.columns.str.replace('[-," "]', '_')
    df=df.drop(df.columns[35], axis=1)

    return df

df=transform(df_all)

C:\Users\samul\AppData\Local\Temp\ipykernel_11532\2009126609.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[-," "]', '_')


In [63]:
df.columns[35]

''

In [49]:
df_all.columns = df_all.columns.str.replace('ä', 'a')
df_all.columns = df_all.columns.str.replace('ö', 'o')

df_all.columns = df_all.columns.str.replace('[-," "]', '_')

C:\Users\samul\AppData\Local\Temp\ipykernel_11532\4037217465.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all.columns = df_all.columns.str.replace('[-," "]', '')


In [65]:
conn=sqlite3.connect('oikotie.db')
c=conn.cursor()

df.to_sql("asunnot", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

In [59]:

with open('sales.txt', 'w') as fp:
    fp.write('\n'.join(columnss))

In [57]:
columns_to_float=['Velaton hinta', 'Myyntihinta', 'Neliöhinta', 'Hoitovastike', 'Yhtiövastike', 'Vesimaksu']

columns=df_all.columns.values 
columnss=[]
for column in columns:
    if column in columns_to_float:
        columnss.append(f'{column} real,')
    else:
        columnss.append(f'{column} text,')
for string in columnss:
    string = string.replace('ä', 'a')
    string = string.replace('ö', 'o')
    string=string.replace('[-," "]', '_')
print(columnss)





['Sijainti text,', 'link text,', 'Kaupunginosa text,', 'Kohdenumero text,', 'Kerros text,', 'Asuinpinta_ala text,', 'Tontin_pinta_ala text,', 'Kokonaispinta_ala text,', 'Pinta_alojen_lisatiedot text,', 'Huoneiston_kokoonpano text,', 'Huoneita text,', 'Kunto text,', 'Lisatietoa_vapautumisesta text,', 'Keittion_varusteet text,', 'Parveke text,', 'Parvekkeen_lisatiedot text,', 'Kylpyhuoneen_varusteet text,', 'Sailytystilat text,', 'Nakymat text,', 'Tulevat_remontit text,', 'Tehdyt_remontit text,', 'Asunnossa_sauna text,', 'Saunan_lisatiedot text,', 'Asumistyyppi text,', 'Palvelut text,', 'Lisatiedot text,', 'Kohde_on text,', 'Velaton_hinta text,', 'Myyntihinta real,', 'Neliohinta text,', 'Hoitovastike real,', 'Yhtiovastike text,', 'Vesimaksu real,', 'Vesimaksun_lisatiedot text,', 'Muut_kustannukset text,', ' text,', 'Uudiskohde text,', 'Taloyhtion_nimi text,', 'Rakennuksen_tyyppi text,', 'Rakennusvuosi text,', 'Huoneistojen_lukumaara text,', 'Kerroksia text,', 'Hissi text,', 'Rakennusmate

In [61]:
print(df.columns)

Index(['Sijainti', 'link', 'Kaupunginosa', 'Kohdenumero', 'Kerros',
       'Asuinpinta_ala', 'Tontin_pinta_ala', 'Kokonaispinta_ala',
       'Pinta_alojen_lisatiedot', 'Huoneiston_kokoonpano',
       ...
       'Ajo_ohjeet', 'Kiinteistotunnus', 'Kiinnitykset', 'Kiinteistovero',
       'Tontin_kuvaus', 'Viemarointi', 'Muut_materiaalit', 'pvm', 'id',
       'status'],
      dtype='object', length=120)


In [ ]:

conn=sqlite3.connect('sqlite:///oikotie.db')
c=conn.cursor()

df_prev=c.execute('''select id, "Velaton hinta" from asunnot''')

sold=df.loc[~df['id'].isin(df_prev['id'])]
price_changed=pd.merge(df, df_prev, on='id')


##set(df.columns).intersection(set(df_prev.columns))
df.to_sql("stg_asunnot", conn, if_exists="replace", index=False)

In [52]:
import sqlite3
import pandas as pd

columnnames='''Sijainti text,
link text,
Kaupunginosa text,
Kohdenumero text,
Kerros text,
Asuinpintaala text,
Tontinpintaala text,
Kokonaispintaala text,
Pintaalojenlisatiedot text,
Huoneistonkokoonpano text,
Huoneita text,
Kunto text,
Lisatietoavapautumisesta text,
Keittionvarusteet text,
Parveke text,
Parvekkeenlisatiedot text,
Kylpyhuoneenvarusteet text,
Sailytystilat text,
Nakymat text,
Tulevatremontit text,
Tehdytremontit text,
Asunnossasauna text,
Saunanlisatiedot text,
Asumistyyppi text,
Palvelut text,
Lisatiedot text,
Kohdeon text,
Velatonhinta text,
Myyntihinta real,
Neliohinta text,
Hoitovastike real,
Yhtiovastike text,
Vesimaksu real,
Vesimaksunlisatiedot text,
Muutkustannukset text,
 text,
Uudiskohde text,
Taloyhtionnimi text,
Rakennuksentyyppi text,
Rakennusvuosi text,
Huoneistojenlukumaara text,
Kerroksia text,
Hissi text,
Rakennusmateriaali text,
Kattomateriaali text,
Kattotyyppi text,
Energialuokka text,
Energiatodistus text,
Kiinteistonantennijarjestelma text,
Tontinkoko text,
Tontinomistus text,
Maanvuokraaja text,
Kiinteistonhoito text,
Isannointi text,
Kaavoitustiedot text,
Kaavatilanne text,
Liikenneyhteydet text,
Kuntotutkimus text,
Lammitys text,
Tontinvuokrapaattyy text,
Tontinvuokra text,
Lisatietojatalosta text,
Pysakointitilankuvaus text,
Yhteisettilat text,
Pintamateriaalit text,
Linkit text,
Rahoitusvastike text,
Saunankustannukset text,
Liiketilojenpintaala text,
Liikehuoneistojenlukumaara text,
Taloyhtiossaonsauna text,
Kerroksienlukumaara text,
Kunnonlisatiedot text,
Keittio text,
Velkaosuus text,
Lammityskustannukset text,
Valmistumisaika text,
Ilmanvaihto text,
Lainaosuudenmaksu text,
Pintaalanlahde text,
Ikkunoidensuunta text,
Vuokrattu text,
Osakeluetteloonsiirrettyhuoneistojarjestelmaan text,
Tiedustelut text,
Autopaikankustannukset text,
Rakennusvuodenlisatiedot text,
Ominaisuudet text,
Sahkoinenomistajamerkinta text,
Lunastuspykala text,
Ilmastointijarjestelma text,
Makuuhuoneenlattia text,
Makuuhuoneenseinat text,
Kylpyhuoneenlattia text,
Kylpyhuoneenseinat text,
Olohuoneenlattia text,
Olohuoneenseinat text,
Vapautuu text,
TVpalvelut text,
Tietoliikennepalvelut text,
Saunamaksu text,
Liikehuoneistojenpintaala text,
Lisatietoahissista text,
Lisatietoataloyhtionsaunasta text,
Kiinteistonlaajakaistayhteys text,
Harrastusmahdollisuudet text,
Rakennuksenlisatiedot text,
Pysakointimaksu text,
Muuttilat text,
Lisatietojamakuuhuoneenvarusteista text,
Olohuoneenvarusteet text,
Ajoohjeet text,
Kiinteistotunnus text,
Kiinnitykset text,
Kiinteistovero text,
Tontinkuvaus text,
Viemarointi text,
Muutmateriaalit text,
pvm text,
id text,
status text'''


conn=sqlite3.connect('oikotie.db')
c=conn.cursor()
c.execute(f''' CREATE TABLE IF NOT EXISTS asunnot({columnnames}
    ) ''')

conn.commit()
conn.close()